In [ ]:
import pandas as pd
import numpy as np
import os
import random
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.metrics import *
from keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Dropout
from tensorflow.keras.layers import LeakyReLU, ELU, PReLU
from tensorflow.keras.metrics import * 
from sklearn.preprocessing import *
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from utils import *
import warnings 
from tqdm import tqdm
warnings.filterwarnings('ignore')

seed = 42
os.environ["PYTHONHASHSEED"] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['TF_CUDNN_USE_FRONTEND '] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = '1' 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)  

In [ ]:
def MLP_regression(X_train, hl_num, bd):   
       
    if isinstance(X_train, pd.DataFrame):
        num = len(X_train.columns)
    elif isinstance(X_train, int) or isinstance(X_train, float): 
        num = X_train   
        
    inputs = Input(shape=(num, ))
    
    if hl_num == 5:
    
        model = Dense(64, kernel_initializer='he_normal')(inputs)   
        if bd == True:
            model = BatchNormalization()(model) 
        model = LeakyReLU()(model)   
        if bd == True:
            model = Dropout(0.2)(model)
        
        model = Dense(32, kernel_initializer='he_normal')(model)   
        if bd == True:  
            model = BatchNormalization()(model)  
        model = LeakyReLU()(model)   
        if bd == True:    
            model = Dropout(0.2)(model)
        
        model = Dense(16, kernel_initializer='he_normal')(model)
        if bd == True:    
            model = BatchNormalization()(model)
        model = LeakyReLU()(model)    
        if bd == True:  
            model = Dropout(0.2)(model)
        
        model = Dense(8, kernel_initializer='he_normal')(model)      
        if bd == True:
            model = BatchNormalization()(model)
        model = LeakyReLU()(model)   
        if bd == True:   
            model = Dropout(0.2)(model)
        
        model = Dense(4, kernel_initializer='he_normal')(model)      
        if bd == True:
            model = BatchNormalization()(model)     
        model = LeakyReLU()(model)
        
        
    elif hl_num == 4:
    
        model = Dense(64, kernel_initializer='he_normal')(inputs)  
        if bd == True:
            model = BatchNormalization()(model) 
        model = LeakyReLU()(model)      
        if bd == True:
            model = Dropout(0.2)(model)
        
        model = Dense(32, kernel_initializer='he_normal')(model)       
        if bd == True:
            model = BatchNormalization()(model)
        model = LeakyReLU()(model)
        if bd == True:
            model = Dropout(0.2)(model)
        
        model = Dense(16, kernel_initializer='he_normal')(model)  
        if bd == True:
            model = BatchNormalization()(model)
        model = LeakyReLU()(model)  
        if bd == True:
            model = Dropout(0.2)(model)
        
        model = Dense(8, kernel_initializer='he_normal')(model)      
        if bd == True:
            model = BatchNormalization()(model)  
        model = LeakyReLU()(model)               
        
        
    elif hl_num == 3:
    
        model = Dense(64, kernel_initializer='he_normal')(inputs)    
        if bd == True:    
            model = BatchNormalization()(model)
        model = LeakyReLU()(model)       
        if bd == True:
            model = Dropout(0.2)(model)  
        
        model = Dense(32, kernel_initializer='he_normal')(model)    
        if bd == True:
            model = BatchNormalization()(model)   
        model = LeakyReLU()(model)  
        if bd == True:     
            model = Dropout(0.2)(model)  
        
        model = Dense(16, kernel_initializer='he_normal')(model)
        if bd == True:
            model = BatchNormalization()(model)
        model = LeakyReLU()(model)
        
    elif hl_num == 2:
    
        model = Dense(64, kernel_initializer='he_normal')(inputs)
        if bd == True:
            model = BatchNormalization()(model)  
        model = LeakyReLU()(model)
        if bd == True:
            model = Dropout(0.2)(model)
        
        model = Dense(16, kernel_initializer='he_normal')(model)
        if bd == True:
            model = BatchNormalization()(model)
        model = LeakyReLU()(model)
        
    elif hl_num == 1:
    
        model = Dense(64, kernel_initializer='he_normal')(inputs)
        if bd == True:
            model = BatchNormalization()(model)
        model = LeakyReLU()(model)
        if bd == True:
            model = Dropout(0.2)(model)
        
        
    output = Dense(1, activation='linear', kernel_initializer='glorot_uniform')(model)

    MLP_model = Model(inputs=[inputs], outputs=[output])
    
    optimizer = Adam(learning_rate=0.001)
   
    MLP_model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mean_absolute_error'])    

    return MLP_model


In [ ]:
train_cv = pd.read_csv('data/train.csv')
train_cv = train_cv.drop('ID', axis = 1)

In [ ]:
kfold = KFold(n_splits=5)

results = {'hl_num': [], 'epochs': [], 'batch_drop': [], 'RMSE': []} 

In [ ]:
for train_idx, test_idx in tqdm(kfold.split(train_cv)):
    
    train, test = train_cv.iloc[train_idx], train_cv.iloc[test_idx] 
    
    train, test = One_hot_encoder(train, test,variable=['Gender', 'Education_Status', 'Employment_Status',
                                                        'Industry_Status', 'Occupation_Status', 'Race',
                                                        'Hispanic_Origin', 'Martial_Status', 'Household_Status',
                                                        'Household_Summary', 'Citizenship', 'Birth_Country',
                                                        'Birth_Country (Father)', 'Birth_Country (Mother)', 'Tax_Status', 'Income_Status'])
    
    train = Drop_only_one_columns(train)
    test = test[train.columns]
       
    train, test = Scaling(train, test)    
    
    X_train, y_train = train.drop('Income', axis=1), train['Income']
    X_test, y_test = test.drop('Income', axis=1), test['Income']

    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train)).batch(128).prefetch(tf.data.AUTOTUNE)    
        
    for hl_num in (range(1, 6)):
        for epochs in [100, 200, 300, 400, 500]:
            for bd in [True, False]:
                model = MLP_regression(X_train, hl_num=hl_num, bd = bd)
                model.fit(train_dataset, epochs=epochs, verbose=1)

                y_pred = model.predict(X_test, verbose=0)                
                
                rmse = np.sqrt(mean_squared_error(y_test, y_pred))       

                results['hl_num'].append(hl_num)
                results['epochs'].append(epochs)
                results['batch_drop'].append(bd)
                
                results['RMSE'].append(rmse)

                del model


results_df = pd.DataFrame(results)
results_df.to_pickle('mlp.csv', index = False)